In [113]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from seaborn import color_palette, JointGrid

filepath = './MHEALTHDATASET/mHealth_subject1.log'

data = pd.read_csv(filepath, sep='\t', skipinitialspace=True, header=None) #parse_dates=[0]
data.columns = ['acc_chest_x', 'acc_chest_y', 'acc_chest_z', 
                'elec_sig_1', 'elec_sig_2', 
                'acc_left_ank_x', 'acc_left_ank_y', 'acc_left_ank_z',
                'gyro_left_ank_x', 'gyro_left_ank_y', 'gyro_left_ank_z',
                'magneto_left_ank_x', 'magneto_left_ank_y', 'magneto_left_ank_z',
                'acc_rig_arm_x', 'acc_rig_arm_y', 'acc_rig_arm_z',
                'gyro_rig_arm_x', 'gyro_rig_arm_y', 'gyro_rig_arm_z',
                'magneto_rig_arm_x', 'magneto_rig_arm_y', 'magneto_rig_arm_z',
                'label']

# print(data.head())
X = data[:].drop('label',axis=1)
Y = data['label']

# print(data.head())
print(data.columns[0])
# del data
# print(X.head())
# print(Y.head())

acc_chest_x


* https://github.com/bhimmetoglu/datasciencecom-mhealth/blob/master/post.md
* http://scjournal.ius.edu.ba/index.php/scjournal/article/view/97/98
* https://arxiv.org/pdf/1809.04356.pdf


- [Python module for machine learning time series](https://github.com/dmbee/seglearn)


In [56]:
from pandas import DataFrame
from pandas import concat
 
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if (type(data) is list or len(data.shape)==1) else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg


values = X[:10].values
data = series_to_supervised(values, 5, 1)
# print(series_to_supervised(Y[:10].values, 5, 1))
# print(data)
# values = Y[:10].values
print(series_to_supervised(values, 5, 1))

   var1(t-5)  var2(t-5)  var3(t-5)  var4(t-5)  var5(t-5)  var6(t-5)  \
5    -9.8184   0.009971    0.29563   0.004186   0.004186     2.1849   
6    -9.8489   0.524040    0.37348   0.004186   0.016745     2.3876   
7    -9.6602   0.181850    0.43742   0.016745   0.037677     2.4086   
8    -9.6507   0.214220    0.24033   0.079540   0.117220     2.1814   
9    -9.7030   0.303890    0.31156   0.221870   0.205130     2.4173   

   var7(t-5)  var8(t-5)  var9(t-5)  var10(t-5)  ...  var14(t)  var15(t)  \
5    -9.6967    0.63077   0.103900    -0.84053  ...  0.010705   -8.7247   
6    -9.5080    0.68389   0.085343    -0.83865  ...  0.294050   -9.0864   
7    -9.5674    0.68113   0.085343    -0.83865  ...  0.144150   -9.0143   
8    -9.4301    0.55031   0.085343    -0.83865  ...  0.153180   -9.0469   
9    -9.3889    0.71098   0.085343    -0.83865  ...  0.297640   -8.8318   

   var16(t)  var17(t)  var18(t)  var19(t)  var20(t)  var21(t)  var22(t)  \
5   -4.0449  0.506090  -0.45686   -1.0082  0.02

In [152]:
from pandas import DataFrame
from pandas import concat
import pandas as pd
import numpy as np

def df_to_supervised_df(df_input, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as Pandas Dataframe.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    columnsOriginal = df_input.columns[:].values
    data = df_input.values
    n_vars = 1 if (len(data.shape)==1) else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('%s(t-%d)' % (columnsOriginal[j], i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('%s(t)' % (columnsOriginal[j])) for j in range(n_vars)]
        else:
            names += [('%s(t+%d)' % (columnsOriginal[j], i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

def df_to_supervised_classification(data, nameClassColumn, n_in_=1, n_out_=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset for a classification problem
    Arguments:
        data: Sequence of observations as Pandas Dataframe.
        nameClassColumn: name of the column with the labels of timeseries 
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    
    X = data[:].drop(nameClassColumn,axis=1)
    Y = data[nameClassColumn]
    
    X = df_to_supervised_df(X, n_in_, n_out_, False)
    X[nameClassColumn] = Y
    
    if dropnan:
        X.dropna(inplace=True)
    return X


df = df_to_supervised_classification(data, 'label', 5, 1)

print(df.shape)

(161275, 139)
